### **Setting GEE environnement**
Due to compatibility issues it is recommended to use a dedicated environment for the geemap library.

See instructions on how to do this here: https://geemap.org/installation/

In [73]:
# Install the libraries if they are not installed (you can remove these lines if you have already installed them)
# !pip install earthengine-api geemap

# Charging the libraries
import ee
import geemap
from datetime import datetime, timedelta

# Authenticate and initialize Earth Engine session
# ee.Authenticate()  # Only required the first time or when the access token expires
ee.Initialize()

### **Looping for export NDVI for each ROI**

In [74]:
# Define the Regions of Interest (ROIs) and the ImageCollections for NDVI and EVI
rois = {
    #'Cotriguacu': ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu_10km'),
    'Guaviare': ee.FeatureCollection('users/caslumali/UE801_Stage/Guaviare_10km'),
    'MDD': ee.FeatureCollection('users/caslumali/UE801_Stage/MDD_10km'),
    #'Paragominas': ee.FeatureCollection('users/caslumali/UE801_Stage/Paragominas_10km'),
}

modisNDVI = ee.ImageCollection('MODIS/061/MOD13Q1').select('NDVI')
modisEVI = ee.ImageCollection('MODIS/061/MOD13Q1').select('EVI')

In [75]:
def export_images(image_collection, roi, roi_name, start_year, end_year, band_name):
    # Get the current datetime to use as a unique identifier
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    folder_name = f'{roi_name}_{band_name}_bimonthly_{timestamp}'

    for year in range(start_year, end_year + 1):
        start_date = ee.Date.fromYMD(year, 7, 31)
        end_date = ee.Date.fromYMD(year + 1, 8, 1)
        date_range = ee.DateRange(start_date, end_date)
        images = image_collection.filterDate(date_range).filterBounds(roi.geometry())

        def export_task(image):
            date = image.date().format('YYYY-MM-dd').getInfo()
            export = ee.batch.Export.image.toDrive(
                image=image.select([band_name]),
                description=f'{roi_name}_{band_name}_{date}',
                folder=folder_name,
                fileNamePrefix=f'{roi_name}_{band_name}_{date}',
                region=roi.geometry(),
                scale=250,
                crs='EPSG:4326'
            )
            export.start()
            print(f'Export task started for {roi_name}, {band_name}, on {date}')

        images_list = images.toList(images.size())
        for i in range(images.size().getInfo()):
            image = ee.Image(images_list.get(i))
            export_task(image)

In [ ]:
# Loop over ROIs
start_year = 2000
end_year = 2022

for roi_name, roi in rois.items():
    print(f'Processing {roi_name}...')
    
    # Export NDVI images
    export_images(modisNDVI, roi, roi_name, start_year, end_year, 'NDVI')

    # Export EVI images
    export_images(modisEVI, roi, roi_name, start_year, end_year, 'EVI')
